## Importações

In [1]:
import os
from itertools import product

import numpy as np
import pandas as pd
from scipy.stats import zscore

from sklearn.datasets import make_regression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.feature_selection import SelectKBest, f_regression, SelectPercentile
import pandas as pd
import numpy as np
from itertools import product
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold, RepeatedKFold, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from scipy.stats import zscore
from sklearn.model_selection import cross_validate


In [2]:
%pip install XlsxWriter


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Plano de Experimentação

### 1. Carregar o dataset

In [3]:
df = pd.read_csv('../../../data/pre-processado/por-municipio/completo/dados-manaus-preprocessado.csv') 

# Retirando antes dos anos 2000 pois dados de clima ausentes, e inserir pela média prejudicou o modelo
df = df[df['ano']>=2000]

# Retirando pois variáveis não necessárias para o modelo, visto que são iguais para todas as linhas
df = df.drop(columns=['municipio','uf'])

### 2. Divisão do Dataset

In [4]:
X = df.drop(columns='vazao')
y = df['vazao'] # Separa o target
cat_cols = X.select_dtypes(include='object').columns.tolist()

### 3. Pré-processamentos

In [5]:
def remover_outliers(df, colunas, threshold=3):
    z_scores = np.abs(zscore(df[colunas]))
    mask = (z_scores < threshold).all(axis=1)
    return df[mask]

### 4. Parâmetros de variação

In [6]:
classificacoes = ['2 classificações', '3 classificações']
validacao = ['k=3', 'k=10']
criterios = ['squared_error']
n_arvores = [50, 100]
etas = [0.1, 0.3]
depths = [3, 5]
normalizadores = [StandardScaler(), MinMaxScaler(), RobustScaler()]
usar_outliers = [True, False]
seletores = [None, SelectKBest(score_func=f_regression, k=5), RFE(estimator=RandomForestRegressor(), n_features_to_select=5)]
validadores = [('k=3', KFold(n_splits=3)), ('k=10', KFold(n_splits=10)), ('rep=5x2', RepeatedKFold(n_splits=5, n_repeats=2))]


### 5. Cenários

In [7]:
cenarios_rfr = list(product(classificacoes, criterios, n_arvores, normalizadores, usar_outliers, seletores, validadores))
cenarios_xgb = list(product(classificacoes, etas, depths, normalizadores, usar_outliers, seletores, validadores))

### 4. Random Forest Regressor

In [ ]:
resultados_rfr = []

for i, (classe, criterio, n, norm, outlier_flag, seletor, (val_tag, validador)) in enumerate(cenarios_rfr):
    X_temp = X.copy()
    y_temp = y.copy()
    num_cols = X_temp.select_dtypes(include=['int64', 'float64']).columns.tolist()

    if outlier_flag:
        df_completo = X_temp.copy()
        df_completo["target"] = y_temp
        df_sem_outliers = remover_outliers(df_completo, num_cols)
        X_temp = df_sem_outliers.drop(columns="target")
        y_temp = df_sem_outliers["target"]

    if X_temp.shape[0] < 2:
        continue

    preprocessador = ColumnTransformer([("scale", norm, num_cols)], remainder='passthrough')
    etapas = [("prep", preprocessador)]

    if seletor:
        etapas.append(("sel", seletor))

    etapas.append(("clf", RandomForestRegressor(n_estimators=n, criterion=criterio, random_state=42)))
    modelo = Pipeline(etapas)

    scores = cross_validate(modelo, X_temp, y_temp, cv=validador, scoring=[
            'neg_mean_squared_error', 'neg_mean_absolute_error', 'r2'
        ])    
    mse = -np.mean(scores['test_neg_mean_squared_error'])
    mae = -np.mean(scores['test_neg_mean_absolute_error'])
    r2 = np.mean(scores['test_r2'])

    resultados_rfr.append([
        i + 1,
        'X' if classe == '2 classificações' else '',
        'X' if classe == '3 classificações' else '',
        criterio,
        n,
        'X' if val_tag == 'k=3' else '',
        'X' if val_tag == 'k=10' else '',
        'X' if outlier_flag else '',
        type(norm).__name__.replace('Scaler', ''),
        type(seletor).__name__ if seletor else "None",
        val_tag,
        mse,
        mae, 
        r2
    ])

### 5. XGBoost Regressor

In [ ]:
resultados_xgbr = []

for i, (classe, eta, depth, norm, outlier_flag, seletor, (val_tag, validador)) in enumerate(cenarios_xgb):
    X_temp = X.copy()
    y_temp = y.copy()
    num_cols = X_temp.select_dtypes(include=['int64', 'float64']).columns.tolist()

    if outlier_flag:
        df_completo = X_temp.copy()
        df_completo["target"] = y_temp
        df_sem_outliers = remover_outliers(df_completo, num_cols)
        X_temp = df_sem_outliers.drop(columns="target")
        y_temp = df_sem_outliers["target"]

    if X_temp.shape[0] < 2:
        continue

    preprocessador = ColumnTransformer([("scale", norm, num_cols)], remainder='passthrough')
    etapas = [("prep", preprocessador)]

    if seletor:
        etapas.append(("sel", seletor))

    etapas.append(("clf", XGBRegressor(learning_rate=eta, max_depth=depth, random_state=42)))
    modelo = Pipeline(etapas)

    scores = cross_validate(modelo, X_temp, y_temp, cv=validador, scoring=[
            'neg_mean_squared_error', 'neg_mean_absolute_error', 'r2'
        ])    
    mse = -np.mean(scores['test_neg_mean_squared_error'])
    mae = -np.mean(scores['test_neg_mean_absolute_error'])
    r2 = np.mean(scores['test_r2'])

    resultados_xgbr.append([
        i + 1,
        'X' if classe == '2 classificações' else '',
        'X' if classe == '3 classificações' else '',
        eta,
        depth,
        'X' if val_tag == 'k=3' else '',
        'X' if val_tag == 'k=10' else '',
        'X' if outlier_flag else '',
        type(norm).__name__.replace('Scaler', ''),
        type(seletor).__name__ if seletor else "None",
        val_tag,
        mse,
        None,
        None
    ])

### 7. Dataframe com Resultados

In [ ]:
df_rfr = pd.DataFrame(resultados_rfr, columns=[
    'Cenário', '2 classificações', '3 classificações', 'Critério',
    'Nº Árvores', 'Validação k=3', 'Validação k=10',
    'Remoção de Outliers', 'Normalização', 'Seleção de Atributos',
    'Validação Cruzada', 'MSE', 'MAE', 'R²'
])

df_xgbr = pd.DataFrame(resultados_xgbr, columns=[
    'Cenário', '2 classificações', '3 classificações', 'Taxa de Aprendizado',
    'Profundidade', 'Validação k=3', 'Validação k=10',
    'Remoção de Outliers', 'Normalização', 'Seleção de Atributos',
    'Validação Cruzada', 'MSE', 'MAE', 'R²'
])

### 7. Formatação do Dataframe

In [ ]:
# XGBoost
df_xgbr['Taxa 0.1'] = df_xgbr['Taxa de Aprendizado'].apply(lambda x: 'X' if x == 0.1 else '')
df_xgbr['Taxa 0.3'] = df_xgbr['Taxa de Aprendizado'].apply(lambda x: 'X' if x == 0.3 else '')
df_xgbr['Profundidade 3'] = df_xgbr['Profundidade'].apply(lambda x: 'X' if x == 3 else '')
df_xgbr['Profundidade 5'] = df_xgbr['Profundidade'].apply(lambda x: 'X' if x == 5 else '')
df_xgbr['Standard'] = df_xgbr['Normalização'].apply(lambda x: 'X' if x == 'Standard' else '')
df_xgbr['MinMax'] = df_xgbr['Normalização'].apply(lambda x: 'X' if x == 'MinMax' else '')
df_xgbr['Robust'] = df_xgbr['Normalização'].apply(lambda x: 'X' if x == 'Robust' else '')

# Random Forest
df_rfr['Standard'] = df_rfr['Normalização'].apply(lambda x: 'X' if x == 'Standard' else '')
df_rfr['MinMax'] = df_rfr['Normalização'].apply(lambda x: 'X' if x == 'MinMax' else '')
df_rfr['Robust'] = df_rfr['Normalização'].apply(lambda x: 'X' if x == 'Robust' else '')

### 8. Dataframes finais

In [ ]:
# Random Forest
df_rfr = df_rfr[[
    'Cenário', '2 classificações', '3 classificações', 
    'Standard', 'MinMax', 'Robust',
    'Critério',
    'Nº Árvores', 'Validação k=3', 'Validação k=10',
    'Remoção de Outliers',
    'Seleção Atributos',
    'Validação Cruzada',
    'MSE', 'MAE', 'R²'
]]

# XGBoost
df_xgbr = df_xgbr[[
    'Cenário', '2 classificações', '3 classificações', 
    'Remoção de Outliers', 
    'Standard', 'MinMax', 'Robust'
    'Taxa 0.1', 'Taxa 0.3', 
    'Profundidade 3', 'Profundidade 5',
    'Validação k=3', 'Validação k=10',
    'Seleção Atributos',
    'Validação Cruzada',
    'MSE', 'MAE', 'R²'
]]

TypeError: list indices must be integers or slices, not list

### 6. Exportação

In [ ]:
with pd.ExcelWriter('../../../data/plano-de-experimentacao/plano_de_experimentacao_7.xlsx', engine='xlsxwriter') as writer:
    df_rfr.to_excel(writer, sheet_name='Random Forest Regressor', index=False)
    df_xgbr.to_excel(writer, sheet_name='XGBoost Regressor', index=False)

print("Excel gerado com sucesso!")

AttributeError: 'list' object has no attribute 'to_excel'

In [ ]:
# import os
# from itertools import product

# import numpy as np
# import pandas as pd
# from scipy.stats import zscore

# from sklearn.datasets import make_regression
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
# from sklearn.model_selection import KFold, cross_val_score
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.feature_selection import SelectKBest, f_regression, SelectPercentile
# from xgboost import XGBRegressor

# # Carregando os dados
# path = '../../../data/pre-processado/por-municipio/completo/dados-manaus-preprocessado.csv'
# df = pd.read_csv(path)
# df = df[df['ano'] >= 2000]
# df = df.drop(columns=['municipio', 'uf'])

# X = df.drop(columns='vazao')
# y = df['vazao']

# cat_cols = X.select_dtypes(include='object').columns.tolist()


# def remover_outliers(df, colunas, threshold=3):
#     z_scores = np.abs(zscore(df[colunas]))
#     mask = (z_scores < threshold).all(axis=1)
#     return df[mask]


# # Configurações para experimentos
# classificacoes = ['2 classificacoes', '3 classificacoes']
# criterios = ['squared_error']
# n_arvores = [50, 100]
# normalizadores = [StandardScaler(), MinMaxScaler(), RobustScaler()]
# usar_outliers = [True, False]

# validacoes = {
#     'k=3': KFold(n_splits=3, shuffle=True, random_state=42),
#     'k=10': KFold(n_splits=10, shuffle=True, random_state=42)
# }

# seletores = {
#     'KBest-3': SelectKBest(score_func=f_regression, k=3),
#     'Percentile-50': SelectPercentile(score_func=f_regression, percentile=50),
#     'Nenhum': 'passthrough'
# }

# cenarios_rfr = list(product(classificacoes, criterios, n_arvores, validacoes.keys(), normalizadores, usar_outliers, seletores.keys()))
# resultados_rfr = []

# for i, (classe, criterio, n, val_key, norm, outlier_flag, sel_key) in enumerate(cenarios_rfr):
#     X_temp = X.copy()
#     y_temp = y.copy()
#     num_cols = X_temp.select_dtypes(include=['int64', 'float64']).columns.tolist()

#     if outlier_flag:
#         df_completo = X_temp.copy()
#         df_completo['target'] = y_temp
#         df_sem_outliers = remover_outliers(df_completo, num_cols)
#         X_temp = df_sem_outliers.drop(columns='target')
#         y_temp = df_sem_outliers['target']

#     if X_temp.shape[0] < 2:
#         continue

#     seletor = seletores[sel_key]
#     steps = []
#     preprocessador = ColumnTransformer([
#         ("scale", norm, num_cols)
#     ], remainder='passthrough')

#     steps.append(('prep', preprocessador))
#     if seletor != 'passthrough':
#         steps.append(('select', seletor))
#     steps.append(('clf', RandomForestRegressor(n_estimators=n, criterion=criterio, random_state=42)))

#     modelo = Pipeline(steps)
#     cv = validacoes[val_key]

#     mse_scores = -1 * cross_val_score(modelo, X_temp, y_temp, cv=cv, scoring='neg_mean_squared_error')
#     mae_scores = -1 * cross_val_score(modelo, X_temp, y_temp, cv=cv, scoring='neg_mean_absolute_error')
#     r2_scores = cross_val_score(modelo, X_temp, y_temp, cv=cv, scoring='r2')

#     resultados_rfr.append([
#         i + 1,
#         'X' if classe == '2 classificacoes' else '',
#         'X' if classe == '3 classificacoes' else '',
#         criterio,
#         n,
#         val_key,
#         'X' if outlier_flag else '',
#         type(norm).__name__.replace('Scaler', ''),
#         sel_key,
#         np.mean(mse_scores),
#         np.mean(mae_scores),
#         np.mean(r2_scores)
#     ])

# # Criar DataFrame final e exportar
# colunas = ['Cenario', '2 classificacoes', '3 classificacoes', 'Criterio', 'N Arvores', 'Validacao',
#            'Remocao de Outliers', 'Normalizacao', 'Feature Selection', 'MSE', 'MAE', 'R2']
# df_rfr = pd.DataFrame(resultados_rfr, columns=colunas)

# with pd.ExcelWriter('../../../data/plano-de-experimentacao/plano_de_experimentacao_cv_fs.xlsx', engine='xlsxwriter') as writer:
#     df_rfr.to_excel(writer, sheet_name='Random Forest FS CV', index=False)

# print("Excel com resultados gerado com sucesso!")


KeyboardInterrupt: 